<a href="https://colab.research.google.com/github/QueeneDelmarva/Thesis/blob/main/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Thesis Code**

## Importing Libraries

In [ ]:
# Connect to Gdrive
from google.colab import drive

# Connect to Gsheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import string
from google.colab import auth
from google.auth import default

## Connecting to Gdrive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Connecting to GSheets

In [ ]:
# Define the scope and create the credentials using the file in your Google Drive
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/skripsi-394804-a98ea9715aa4.json', scope)

# Authenticate with gspread
client = gspread.authorize(creds)

# Open a specific Google Sheet by its title or URL
sheet = client.open_by_key('1RlZ8-XwwEueuCAq4nXGwq3ZmPDE-_0gP3PsroKeNFGo').sheet1


# ## Read the data from the Gsheets
# # 1. Read individual cells
# cell_value = sheet.cell(3, 5).value
# print(cell_value)

# # 2. Read all values from the first worksheet
# data = sheet.get_all_values()

# # Print the data
# for row in data:
#     print(row)

## **Radix Trie**
---
Classes:
*   Node
*   Function

### Radix Trie Node

In [ ]:
# Defines a class representing a single node in the radix trie, holding information about its children and whether it marks the end of a word.
class RadixTrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

### Radix Trie Function

In [ ]:
# Defines a class that implements a radix trie data structure, allowing efficient storage and retrieval of a set of strings with common prefixes.
class RadixTrie:
    def __init__(self):
        self.root = RadixTrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = RadixTrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def traverse(self, node=None, prefix=""):
        if node is None:
            node = self.root

        if node.is_end_of_word:
            words = prefix.split()  # Split the phrase into individual words
            for word in words:
                print("Word:", word)

        for char, child in node.children.items():
            self.traverse(child, prefix + char)

### **Main Function**

In [ ]:
def main():
    # Read all values from the first worksheet
    data = sheet.get_all_values()

    # Build the Radix Trie from the data
    trie = RadixTrie()
    for row in data:
        for word in row:
            trie.insert(word)

    # Traverse and print the trie nodes and their children
    print("Nodes and Their Children:")
    trie.traverse()

if __name__ == "__main__":
    main()

Nodes and Their Children:
Word: Date
Word: From
Word: To
Word: Subject
Word: Content
Word: 01/02/2023
Word: Raymond
Word: Koh
Word: Re:
Word: Ground
Word: Transport
Word: Request
Word: -
Word: T7977
Word: -
Word: FS0012023
Word: -
Word: Wed
Word: 01
Word: Feb
Word: 2023
Word: 0840
Word: LT
Word: ACAM
Word: OCC
Word: Noted
Word: and
Word: confirmed
Word: thanks.
Word: Ground
Word: Transport
Word: Request
Word: -
Word: T7977
Word: -
Word: FS0012023
Word: -
Word: Wed
Word: 01
Word: Feb
Word: 2023
Word: 0840
Word: LT
Word: Ground
Word: Transport
Word: Request
Word: -
Word: T7977
Word: -
Word: FS0012023
Word: -
Word: Tue
Word: 31
Word: Jan
Word: 2023
Word: 1930LT
Word: Please
Word: arrange
Word: Ground
Word: Transportation
Word: as
Word: follows:
Word: Request
Word: Date:
Word: Wed
Word: 01
Word: Feb
Word: 2023
Word: Request
Word: Time:
Word: QG-552
Word: ETA
Word: 0840LT
Word: CHANGI
Word: AIRPORT
Word: Drop-off
Word: Location:
Word: SELETAR
Word: BUSINESS
Word: AVIATION
Word: CENTRE21
Wor